In [ ]:
import numpy as np
import os
import pandas as pd
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import datasets
from bids import BIDSLayout

# --- 1. Setup Layout and Groups ---
data_path = '/Volumes/T9/ds001486/derivatives/fmriprep'
layout = BIDSLayout(data_path, validate=False, derivatives=False)
bold_files = layout.get(suffix='bold', extension='nii.gz', desc='preproc', return_type='file')

# Your predefined groups from your research
mld_subs = ['059', '065', '067', '069', '071', '075', '076', '077', '078', '083', '088', '095', '096', '103', '106']
td_subs = ['090', '036', '013', '008', '057', '070', '023', '024', '053', '044', '034', '060', '007', '027', '010']

# --- 2. Setup Atlas and GNN-Specific Masker ---
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=7)
masker = NiftiLabelsMasker(labels_img=atlas.maps, standardize='zscore_sample', memory=None)

# IMPORTANT: vectorize=False keeps the 400x400 shape for GNNs
conn_measure = ConnectivityMeasure(kind='correlation', vectorize=False, discard_diagonal=False, standardize='zscore_sample')

X_raw = []          
y_group = []    
y_task = []     
subject_ids = []     

output_dir = os.path.expanduser('~/Desktop/math_results_gnn_raw')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Starting raw matrix extraction...")

for bold_path in bold_files:
    filename = os.path.basename(bold_path)
    
    # Task Filtering (Only Math tasks)
    if 'task-Mult' in filename:
        current_task = 1
    elif 'task-Sub' in filename:
        current_task = 0
    else:
        continue 

    # Confound Handling
    parts = filename.split('_')
    essential_parts = [p for p in parts if any(x in p for x in ['sub-', 'ses-', 'task-', 'run-'])]
    confound_name = "_".join(essential_parts) + "_desc-confounds_timeseries.tsv"
    confound_path = os.path.join(os.path.dirname(bold_path), confound_name)

    if not os.path.exists(confound_path):
        continue

    sub_id = filename.split('_')[0].split('-')[1] 
    group_label = 1 if sub_id in mld_subs else 0 
    
    try:
        df = pd.read_csv(confound_path, sep='\t')
        df_clean = df.select_dtypes(include=[np.number]).fillna(0).dropna(axis=1, how='all')
        
        time_series = masker.fit_transform(bold_path, confounds=df_clean)
        
        if np.any(np.isnan(time_series)): continue
            
        # This creates a 400x400 matrix
        matrix = conn_measure.fit_transform([time_series])[0]
        
        X_raw.append(matrix)
        y_group.append(group_label)
        y_task.append(current_task) 
        subject_ids.append(sub_id) 
        
        print(f"Processed: sub-{sub_id} | Task: {'Mult' if current_task==1 else 'Sub'}")
        
    except Exception as e:
        print(f"Error on sub-{sub_id}: {e}")

# --- 3. Final Save of Raw Data ---
X_final = np.array(X_raw)
y_final = np.array(y_group)
t_final = np.array(y_task)
s_final = np.array(subject_ids)

# Save everything to your Desktop folder
np.save(os.path.join(output_dir, 'X_raw_gnn.npy'), X_final)    # Shape: (N_scans, 400, 400)
np.save(os.path.join(output_dir, 'y_labels.npy'), y_final)     # MLD (1) vs TD (0)
np.save(os.path.join(output_dir, 'y_tasks.npy'), t_final)      # Mult (1) vs Sub (0)
np.save(os.path.join(output_dir, 'subject_ids.npy'), s_final)  # Tracking which scan belongs to whom

print(f"\nSUCCESS: Saved {len(X_final)} raw matrices to {output_dir}")

[fetch_atlas_schaefer_2018] Dataset found in /Users/jchong058/nilearn_data/schaefer_2018
Starting raw matrix extraction...


/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:63: DeprecationWarning: From release 0.14.0, confounds will be standardized using the sample std instead of the population std.
  time_series = masker.fit_transform(bold_path, confounds=df_clean)
/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:68: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.14.0, the default strategy will be replaced by the new strategy, the 'zscore' option will be removed. and using standardize=True will fall back to 'zscore_sample'.To avoid this warning, please use 'zscore_sample' instead.
  matrix = conn_measure.fit_transform([time_series])[0]


Processed: sub-007 | Task: Mult


/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:63: DeprecationWarning: From release 0.14.0, confounds will be standardized using the sample std instead of the population std.
  time_series = masker.fit_transform(bold_path, confounds=df_clean)
/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:68: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.14.0, the default strategy will be replaced by the new strategy, the 'zscore' option will be removed. and using standardize=True will fall back to 'zscore_sample'.To avoid this warning, please use 'zscore_sample' instead.
  matrix = conn_measure.fit_transform([time_series])[0]


Processed: sub-007 | Task: Mult


...

/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:63: DeprecationWarning: From release 0.14.0, confounds will be standardized using the sample std instead of the population std.
  time_series = masker.fit_transform(bold_path, confounds=df_clean)
/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:68: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.14.0, the default strategy will be replaced by the new strategy, the 'zscore' option will be removed. and using standardize=True will fall back to 'zscore_sample'.To avoid this warning, please use 'zscore_sample' instead.
  matrix = conn_measure.fit_transform([time_series])[0]


Processed: sub-106 | Task: Mult


/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:63: DeprecationWarning: From release 0.14.0, confounds will be standardized using the sample std instead of the population std.
  time_series = masker.fit_transform(bold_path, confounds=df_clean)
/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:68: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.14.0, the default strategy will be replaced by the new strategy, the 'zscore' option will be removed. and using standardize=True will fall back to 'zscore_sample'.To avoid this warning, please use 'zscore_sample' instead.
  matrix = conn_measure.fit_transform([time_series])[0]


Processed: sub-106 | Task: Sub
Processed: sub-106 | Task: Sub

SUCCESS: Saved 240 raw matrices to /Users/jchong058/Desktop/math_results_gnn_raw


/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:63: DeprecationWarning: From release 0.14.0, confounds will be standardized using the sample std instead of the population std.
  time_series = masker.fit_transform(bold_path, confounds=df_clean)
/var/folders/nb/t9_yz5pn7ql2nyvm9cy5kngc0000gn/T/ipykernel_5409/1735230471.py:68: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.14.0, the default strategy will be replaced by the new strategy, the 'zscore' option will be removed. and using standardize=True will fall back to 'zscore_sample'.To avoid this warning, please use 'zscore_sample' instead.
  matrix = conn_measure.fit_transform([time_series])[0]
